In [ ]:
%%HTML
<style type="text/css">
div.h1 {
    background-color:#eebbcb; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 35px; 
    max-width: 1500px; 
    margin: auto; 
    margin-top: 50px;
}

div.h2 {
    background-color:#2ca9e1; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 35px; 
    max-width: 1500px; 
    margin: auto; 
    margin-top: 50px;
}
</style>

**I refered the folloing great notebook for data pipeline:**

https://www.kaggle.com/fchmiel/xgboost-baseline-multilabel-classification



## If you like, please Upvote😹

# <div class="h1">About this notebook</div>

## In this notebook, I create HistGradientBoostingRegressor baseline.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html

### <u>Note</u>

**Update information**

version1: First release. It doesn't seem to be performing well enough, so I will be tuning it up.

version2: Add PCA dimension reduction.

version3: Tuned learing rate.

version4: Tuned max_depth.

version5: Tuned l2_regularization and , max_iter.

version6: Change kfold 5 -> 10.

# <div class="h2">Baseline</div>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor


import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputRegressor

import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
SEED = 42
NFOLDS = 10
DATA_DIR = '/kaggle/input/lish-moa/'
np.random.seed(SEED)

### load data

In [ ]:
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')

test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [ ]:
train.dtypes

In [ ]:
targets.dtypes

### PCA

I refered this notebook https://www.kaggle.com/namanj27/new-baseline-pytorch-moa.

In [ ]:
GENES = [col for col in train.columns if col.startswith('g-')]
CELLS = [col for col in train.columns if col.startswith('c-')]

In [ ]:
# GENES
n_comp = 50

data = pd.concat([pd.DataFrame(train[GENES]), pd.DataFrame(test[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train.shape[0]]; test2 = data2[-test.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train = pd.concat((train, train2), axis=1)
test = pd.concat((test, test2), axis=1)

In [ ]:
#CELLS
n_comp = 15

data = pd.concat([pd.DataFrame(train[CELLS]), pd.DataFrame(test[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train.shape[0]]; test2 = data2[-test.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train = pd.concat((train, train2), axis=1)
test = pd.concat((test, test2), axis=1)

In [ ]:
train = train.drop(columns = (GENES + CELLS))
test = test.drop(columns = (GENES + CELLS))

### Scaling and Encording

In [ ]:
#One-hot encoding for category variables
all_x = pd.concat([train, test])
all_x = pd.get_dummies(all_x, columns=["cp_type", "cp_dose"])

#Adjust scale
all_x["cp_time"] = all_x["cp_time"]/72

#Split into the original dataset.
train = all_x.iloc[:train.shape[0], :].reset_index(drop=True)
test = all_x.iloc[train.shape[0]:, :].reset_index(drop=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# drop id col
X = train.iloc[:,1:].to_numpy()
X_test = test.iloc[:,1:].to_numpy()
y = targets.iloc[:,1:].values

### Model

In [ ]:
reg = HistGradientBoostingRegressor(learning_rate = 0.013, max_depth=5, l2_regularization=0.02,
                                    max_iter=175)
reg = MultiOutputRegressor(reg)


### Train the imprement

In [ ]:
oof_preds = np.zeros(y.shape)
test_preds = np.zeros((test.shape[0], y.shape[1]))
oof_losses = []
#kf = KFold(n_splits=NFOLDS)
kf = MultilabelStratifiedKFold(n_splits=NFOLDS)
for fn, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print('Starting fold: ', fn)
    X_train, X_val = X[trn_idx], X[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]

    # drop where cp_type==ctl_vehicle (baseline)
    ctl_mask = X_train[:,-4]== 1#'ctl_vehicle'
    X_train = X_train[~ctl_mask,:]
    y_train = y_train[~ctl_mask]
    

    
    reg.fit(X_train, y_train)
    val_preds = reg.predict(X_val) # list of preds per class
    #val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    oof_losses.append(loss)
    preds = reg.predict(X_test)
    #preds = np.array(preds)[:,:,1].T # take the positive class
    test_preds += preds / NFOLDS
    
print(oof_losses)
print('Mean OOF loss across folds', np.mean(oof_losses))
print('STD OOF loss across folds', np.std(oof_losses))

### Validation

In [ ]:
# set control train preds to 0
control_mask = train['cp_type_ctl_vehicle']==1
oof_preds[control_mask] = 0

print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(oof_preds)))

## Save OOF preds


In [ ]:
test[sub.columns[1:]] = test_preds

In [ ]:
test.head()

In [ ]:
sub = sub.drop(columns=sub.columns[1:]).merge(test[list(sub.columns[1:].values) + ["sig_id"]], on='sig_id', how='inner')

In [ ]:
sub[sub.columns[1:]] = sub[sub.columns[1:]].clip(0, 1, axis=1)

In [ ]:
sub

In [ ]:
sub.to_csv('submission.csv', index=False)